In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# 정확한 chromedriver 경로 설정
chrome_service = Service('/opt/homebrew/bin/chromedriver')  # 실제 경로로 업데이트
chrome_options = Options()

d = webdriver.Chrome(service=chrome_service, options=chrome_options)
d.implicitly_wait(3)

# 페이지 불러오기 및 탐색
d.get('https://korean.visitkorea.or.kr/search/search_list.do?keyword=%EB%93%9C%EB%9D%BC%EC%9D%B4%EB%B8%8C')

# '여행코스' 클릭
d.find_element(By.XPATH, '//*[@id="t_course"]').click()
time.sleep(2)

# '페이지' 클릭
d.find_element(By.XPATH, '//*[@id="search_result"]/ul/li[1]/div[1]/div[1]/a').click()
time.sleep(2)

# JavaScript 실행하여 h2 요소에서 em 요소를 제외한 텍스트 가져오기
script = '''
    var h2Element = document.querySelector('#contents > div:nth-child(1) > div:nth-child(2) > h2');
    var emElement = h2Element.querySelector('em');
    if (emElement) emElement.remove();
    return h2Element.innerText;
'''
title = d.execute_script(script)

print(title)  

element = d.find_element(By.XPATH, '//*[@id="dist"]')
distance = element.text
print(distance)

element = d.find_element(By.XPATH, '//*[@id="contents"]/div[2]/div[2]/div/div[1]/div/p')
explain = element.text
print(explain)


# "더보기" 버튼이 있는 경우 클릭하여 모든 코스를 로드
while True:
    try:
        more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.swiper-button-next')))
        more_button.click()
        time.sleep(2)  # 더보기 버튼 클릭 후 로딩 대기
    except:
        break  # 더 이상 "더보기" 버튼이 없으면 반복 종료

# 코스 리스트 가져오기
course_list = d.find_elements(By.CSS_SELECTOR, 'ul.cosList > li')

processed_courses = set()

for index, course in enumerate(course_list, start=1):
    # 각 코스 클릭
    course_link = course.find_element(By.TAG_NAME, 'a')
    tab_id = course_link.get_attribute('href').split('#')[-1]

    if tab_id in processed_courses:
        continue  # 이미 처리된 코스는 건너뛰기

    d.execute_script("arguments[0].click();", course_link)  # JavaScript로 클릭
    time.sleep(2)  # 탭 전환 후 로딩 대기

    # 정보 크롤링
    element = WebDriverWait(d, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="{tab_id}"]/div/div[1]/div/em')))
    order = element.text
    print(f"Order: {order}")

    element = d.find_element(By.XPATH, f'//*[@id="{tab_id}"]/div/div[1]/div/strong/a')
    destination = element.text
    print(f"Destination: {destination}")

    element = d.find_element(By.XPATH, f'//*[@id="{tab_id}"]/div/div[1]/span[1]')
    address = element.text
    print(f"Address: {address}")

    element = d.find_element(By.XPATH, f'//*[@id="{tab_id}"]/div/ul')
    tag = element.text
    print(f"Tag: {tag}")

    processed_courses.add(tab_id)  # 현재 코스를 처리된 목록에 추가

# 웹드라이버 종료
d.quit()

여수 오션뷰 힐링 드라이브 코스
코스 총거리 : 8.6km
전라남도 대표 미항(美港)인 여수에서는 다양한 매력의 바다를 즐길 수 있다.
울창한 마래산을 지나 남해와 마주보는 바다를 끼고 힐링을 즐길 수 있는 드라이브 코스로 3가지의 매력을 가진 해변을 만나보자.
Order: 1
Destination: 미평산림욕장
Address: 전라남도 여수시 미평동 21-1
Tag: #관광지
#산림욕
#숲산책
#여수관광
#여수여행
#전남여행
Order: 2
Destination: 여수 녹테마레
Address: 전라남도 여수시 만성로 (294)
Tag: #가볼래터
#문화시설
#미디어아트
#여수
#여수가볼만한곳
#여행구독
#전시관
Order: 3
Destination: 여수국가대표패러글라이딩
Address: 전라남도 여수시 망양로 225
Tag: #레포츠
#여수시
#여수패러글라이딩
#패러글라이딩
Order: 4
Destination: 여수 해양레일바이크
Address: 전라남도 여수시 망양로 187
Tag: #2024_여행가는달과함께하는_발도장여행지
#가족과함께
#레일바이크
#레포츠
#여수가볼만한곳
#여수바다
#여수여행
#여수해양레일바이크
#연인과함께
#친구와함께
#해양레일바이크
Order: 5
Destination: 만성리 검은모래해변
Address: 전라남도 여수시 만성리길 15-1
Tag: #가족과함께
#검은모래해변
#관광지
#바다
#아이와함께
#여수가볼만한곳
#여수검은모래사장
#여수모래찜질명소
#여수바닷가
#여수여행
#여수해수욕장추천
#자연
#자연여행
#자연좋은곳
#자연풍경
#자연환경
#전라권
#전라남도여행
#휴식여행
#휴식하기좋은곳
Order: 6
Destination: 모사금해수욕장
Address: 전라남도 여수시 오천동
Tag: #가족과함께
#관광지
#모사금피서지
#모사금해수욕장
#여름피서지
#여름휴가
#자연
#자연좋은곳
#자연환경
#전라권
#풍경좋은곳
#해수욕장
#휴식여행
#휴식하기좋은곳
#힐링여행
Order: 7
Destination: 신덕해수욕장 / 신덕해변
